
# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap
import pickle

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.preprocessing import LabelEncoder

from PositionalModelLinear import PositionalModelLinear

from config import *
from src.match_names import name_fbref_to_fpl, neutralize_name

# Loading data
Data consists of all PL players logs for 2021-22 and 2022-23 seasons\
A single log is just a summary of player performance in a particular match

In [3]:
# loading csv
df_original = pd.read_csv('data/previous/FPL_logs.csv')

df = df_original.copy()

In [4]:
df_original.columns

Index(['Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name', 'Start', 'Pos',
       'Min', 'Gls',
       ...
       'xG_team_15', 'xGA_team_15', 'xG_opp_15', 'xGA_opp_15', 'xG_team_30',
       'xGA_team_30', 'xG_opp_30', 'xGA_opp_30', 'xG_diff', 'Avg_FPL_points'],
      dtype='object', length=114)

In [5]:
df_original["Name"].unique().size

544

In [6]:
df.shape

(19208, 114)

In [7]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [8]:
# df = df[~df["FPL_pos"].isin(["GK"])]

In [9]:
df["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [10]:
# df = df[df["Season"].isin(['2021-22', '2022-23'])]
allowed_pos = ['FWD', "MID", "DEF", "GK"]
df = df[df["FPL_pos"].isin(allowed_pos)]

In [11]:
df = df.sort_values(by=["Name", "Date"])

# Feature selection
Model will use a set of standard features and moving averages for some other feature values. In this case we will use moving averages for last 2, 4 and 30 gameweeks.

In [12]:
rolling_gameweeks = [4]
# rolling_gameweeks = [5]

# to_predict = "FPL_points"
to_predict = "xP"
standard_features = [
             "Was_home", # Home/Away
             "Team_rating", "Opp_rating", 
             "Rating_difference", # Team ratings
             # "Price", # FPl price
             # "Transfers_balance",
             # "Transfers_result"
             # "Avg_FPL_points",
             # "Avg_xP",
             # 'Avg_FPL_points_venue', 
             # "Was_home_xP",
             "RD_xP",
             # "Team_xP",
             # "Opp_xP"
             "xG_team_4",
             # "xGA_team_4",
             # "xG_opp_4",
             "xGA_opp_4"
             ]
features_to_roll = [
                # "Min", 
                # "Start", # time played
                # 'Gls', 
                # 'Sh', 
                # 'SoT', # Goals
                # 'Ast', # Assists
                # 'CrdY', 'CrdR', # Cards
                # "xG", 'xA', # Expected
                # 'Team_CS', # Defence
                # 'Team_score', 'Opp_score', 'Team_result', # Team form
                # "Cmp%", "PrgP", "PrgC", "T_succ",
                # 'Bonus', 'Bps', # Bonus
                # 'ICT_index', # ICT
                # "FPL_points", 
                "xP"
                # "Baseline_points", "Bonus" # FPL points
            ]
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "Price", "FPL_pos", "FPL_points"]

In [13]:
# df[df["Name"] == "Erling-Haaland"]

# Feature engineering

In [14]:
df["Rating_difference"] = df["Team_rating"] / df["Opp_rating"]
df["Baseline_points"] = df["FPL_points"] - df["Bonus"]
df["Transfers_result"] = df["Transfers_balance"] >= 0

In [15]:
def add_linear_fixtures(df):
    RD_coef_gk = 0.08
    RD_coef_outfield = 0.37
    df["Was_home_xP"] = df["Avg_FPL_points"] * df["Was_home"]
    df.loc[df["FPL_pos"] == "GK", "RD_xP"] = df["Avg_FPL_points"] * ( 1 + (df["Rating_difference"] - 1) * RD_coef_gk )
    df.loc[df["FPL_pos"] != "GK", "RD_xP"] = df["Avg_FPL_points"] * ( 1 + (df["Rating_difference"] - 1) * RD_coef_outfield )
    df["Team_xP"] = df["Avg_FPL_points"] * df["Team_rating"]
    df["Opp_xP"] = df["Avg_FPL_points"] * df["Opp_rating"]
    df['Avg_xP'] = df.groupby('Name_original')['xP'].transform('mean')
    df.loc[df["Was_home"] == True, 'Avg_FPL_points_venue'] = df[df["Was_home"] == True].groupby('Name_original')['FPL_points'].transform('mean')
    df.loc[df["Was_home"] == False, 'Avg_FPL_points_venue'] = df[df["Was_home"] == False].groupby('Name_original')['FPL_points'].transform('mean')
    
    return df

In [16]:
df = add_linear_fixtures(df)

In [17]:
def calculate_team_points(row):
    # win - 3 points
    if row['Team_score'] > row['Opp_score']:
        return 3
    # draw - 1 point
    elif row['Team_score'] == row['Opp_score']:
        return 1
    # loss - 0 points
    else:
        return 0
    
df['Team_result'] = df.apply(calculate_team_points, axis=1)

In [18]:
def add_rolling_features(df, standard_features, features_to_roll):
    features = standard_features
    
    for r in rolling_gameweeks:
        form_means = df.groupby(["Name"])[features_to_roll].rolling(r, min_periods=1).mean().groupby(["Name"]).shift(1).reset_index()
        # print(form_means[form_means["Name"] == "Erling-Haaland"])
        form_means = form_means.fillna(method='bfill') # slightly incorrect, better to drop Nan
        form_means.columns = [f'{col}{"_"}{r}' for col in form_means.columns]
        features += form_means.columns.tolist()
        features = list(filter(lambda x: x not in ["Name_" + str(r)], features))
        df = pd.concat([df.reset_index(), form_means], axis=1)
        # df = df.merge(form_means, left_index=True, right_index=True)
        df = df.drop([col for col in df.columns if col.startswith('level')], axis=1)
        
    return df.reset_index(), features

In [19]:
def ohe(df, ohe_columns, features):
    # one hot encoding
    for c in ohe_columns:
        ohe_c = pd.get_dummies(df[c], dtype="int64")
        df = pd.concat([df, ohe_c], axis=1)

        features += ohe_c.columns.tolist()
        
    return df, features

In [20]:
def label_encoding(df, column_to_encode):
    
    mapping_dict = {
        'Y': 1, # Starting eleven
        'Y*': 1, # Starting eleven as captain
        'N': 0, # Not in starting eleven
    }
    
    df[column_to_encode] = df[column_to_encode].map(mapping_dict)
    
    return df

In [21]:
df = label_encoding(df, "Start")

In [22]:
df, features = add_rolling_features(df, standard_features, features_to_roll)

In [23]:
# df, features = ohe(df, ["FPL_pos"], features)
# if "GK" in df.columns:
#     df = df.drop("GK", axis=1)
#     features.remove("GK")

In [24]:
# dropping unwanted columns
features = [col for col in features if not col.startswith('level')]
df = df[np.unique(features + info + [to_predict])]

In [25]:
df = df[pd.to_numeric(df["GW"], errors="coerce").notna()]
df["GW"] = df["GW"].astype("uint64")

In [26]:
# dropping NaNs
df = df.dropna(axis=0)

In [27]:
# df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [28]:
df.columns

Index(['FPL_points', 'FPL_pos', 'GW', 'Name_original', 'Opp_rating',
       'Opponent', 'Price', 'RD_xP', 'Rating_difference', 'Season', 'Team',
       'Team_rating', 'Was_home', 'xGA_opp_4', 'xG_team_4', 'xP', 'xP_4'],
      dtype='object')

In [29]:
df.shape

(18722, 17)

In [30]:
# df = df[~df["FPL_pos"].isin(["GK"])] # only outfield players

In [31]:
df.shape

(18722, 17)

# Data split into train and valid
Let's take entire 2021-22 season and 30 first gameweeks of 2022-23 season as training data and 8 last gameweeks of that season as valid data (~10% of all rows).

In [32]:
def shrink_df_to_top_players(df, n_players, min_fixtures):
    # Calculate average scores
    average_scores = df.groupby('Name_original')[to_predict].mean()
    
    # Calculate player counts
    player_counts = df['Name_original'].value_counts()
    
    # Filter out players with less than min_fixtures occurrences
    popular_players = player_counts[player_counts >= min_fixtures].index
    
    print("Unique players with min_fixtures:", popular_players.size)

    # Sort players by average scores
    sorted_players = average_scores.sort_values(ascending=False)

    # Get the top n_players players
    top_popular_players = sorted_players[sorted_players.index.isin(popular_players)].head(n_players)

    # Filter the original DataFrame
    top_players_df = df[df['Name_original'].isin(top_popular_players.index)]
    
    print("Unique players left:", top_players_df['Name_original'].unique().size)
    
    return top_players_df

In [33]:
CUT_OFF_GAMEWEEK = 31
SEASON_TO_PREDICT = "2022-23"

In [34]:
df_train = shrink_df_to_top_players(df, 550, 10)
# df_train["Name_original"].unique()

Unique players with min_fixtures: 390
Unique players left: 390


In [35]:
df = df.set_index(["Name_original", "Season", "GW"], drop=False)

In [36]:
# training data
X_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)
y_train = df_train[((df_train["Season"] < SEASON_TO_PREDICT) | (df_train["GW"] <= CUT_OFF_GAMEWEEK - 1))].reset_index(drop=True)

In [37]:
# only 31st gameweek
X_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [38]:
# all remaining gameweeks
X_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)
y_test_all_remaining = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) ].reset_index(drop=True)

In [39]:
X_train.shape, X_test.shape, X_test_all_remaining.shape

((16104, 17), (273, 17), (2213, 17))

# Model

In [40]:
# model = GradientBoostingRegressor(random_state=42)
# model = XGBRegressor(
#         random_state=42,
#         n_estimators=500,
#         # early_stopping_rounds=5,
#         learning_rate=0.2
#     )
model = PositionalModelLinear(features, features, [], to_predict)
# model = RandomForestRegressor(random_state=42)

In [41]:
%%time
model.fit(
        X_train, 
        y_train,
        # eval_set=[(X_test_all_remaining, y_test_all_remaining)],
        # verbose=False
    )

CPU times: total: 31.2 ms
Wall time: 42 ms


In [42]:
features

['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'RD_xP',
 'xG_team_4',
 'xGA_opp_4',
 'xP_4']

In [43]:
# model.model_GK.feature_names_in_

In [44]:
# model.model_GK.coef_

In [45]:
# model.model_outfield.coef_

# Getting predictions

In [46]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    if all_remaining:
        df_predictions = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
    else:
        df_predictions = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    
    preds = df_predictions[info + [to_predict] + ["Pred"]].sort_values(by=["Pred"], ascending = False)
    
    return preds, predictions

In [47]:
preds, predictions = get_predictions(model, df, X_test)

In [48]:
preds_all_gameweeks, predictions_all = get_predictions(model, df, X_test_all_remaining, True)

# Custom metric
Such models are usually used to compare players and decide who to pick for upcoming gameweek(s). For each player pair in a subset of players model can either predict the outcome well (e.g. player A scores more than player B and model predicts exactly that) or predict wrongly (e.g. player A scores more than player B but model predicts player B > player A). Pairwise_accuracy returns the percentage of corrected predicted pairs. Pairwise_accuracy_topX is a variation of this metric calculated only for X highest scoring players of last two seasons.

In [49]:
def get_top_performer_names(df, no_top, no_gws):
    # takes no_top players that recorded highest average FPL points in no_gws last gameweeks
    return df.reset_index(drop=True).groupby("Name_original")[to_predict].mean().groupby("Name_original").tail(no_gws).sort_values(ascending=False).head(no_top).index.to_list()

In [50]:
def pairwise_accuracy(predicted_scores, true_scores):
    if len(predicted_scores) != len(true_scores):
        raise ValueError("The length of predicted_scores and true_scores must be the same.")

    num_pairs = 0
    num_correct_pairs = 0

    for i in range(len(predicted_scores)):
        for j in range(i + 1, len(predicted_scores)):
            # Check if the predicted order matches the true order
            if (predicted_scores[i] > predicted_scores[j] and true_scores[i] > true_scores[j]) or \
               (predicted_scores[i] < predicted_scores[j] and true_scores[i] < true_scores[j]):
                num_correct_pairs += 1
            num_pairs += 1

    pairwise_accuracy = num_correct_pairs / num_pairs
    return pairwise_accuracy

In [51]:
def pairwise_accuracy_topX(model, df, top_x, all_gw=False):
    # pairwise_accuracy for top_X players
    if all_gw:
        top_performers = df[(df["GW"] == CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]
    else:
        top_performers = df[(df["GW"] >= CUT_OFF_GAMEWEEK) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name_original"].isin(get_top_performer_names(df, top_x, 50)))]

    X = top_performers.reset_index(drop=True)
    y = np.array(top_performers[to_predict].reset_index(drop=True))
    preds = model.predict(X)
    
    return  pairwise_accuracy(preds, y)

# Evaluation

In [52]:
def evaluate(model, df, predictions, y_true, all_gw=False):
    # function to calculate different metrics for a given model
    mae = mean_absolute_error(y_true[to_predict], predictions)
    mse = mean_squared_error(y_true[to_predict], predictions)

    pairwise_acc = pairwise_accuracy(np.array(y_true[to_predict]), predictions)
    pairwise_accuracy_top20 = pairwise_accuracy_topX(model, df, 20, all_gw)
    pairwise_accuracy_top100 = pairwise_accuracy_topX(model, df, 100, all_gw)
    
    print("MAE:", mae)
    print("MSE:", mse)
    print("Pairwise accuracy:", pairwise_acc)
    print("Pairwise accuracy @TOP100:", pairwise_accuracy_top100)
    print("Pairwise accuracy @TOP20:", pairwise_accuracy_top20)

In [53]:
# df[(df["GW"] == GAMEWEEK_TO_PREDICT) & (df["Season"] == SEASON_TO_PREDICT) & (df["Name"].isin(get_top_performer_names(df, 20, 50)))].columns

In [54]:
# next gameweek
# evaluate(model, df, predictions, y_test, all_gw=False)

In [55]:
# all remaining gameweeks
evaluate(model, df, predictions_all, y_test_all_remaining, all_gw=True)

MAE: 1.5387337880901875
MSE: 4.517793557891326
Pairwise accuracy: 0.6430303753343101
Pairwise accuracy @TOP100: 0.584561403508772
Pairwise accuracy @TOP20: 0.7727272727272727


In [56]:
pred_sum = preds_all_gameweeks["Pred"].sum()
fpl_sum = preds_all_gameweeks["FPL_points"].sum()
xp_sum = preds_all_gameweeks["xP"].sum()

pd.DataFrame([pred_sum, fpl_sum, xp_sum], index=["Pred", "FPL", "xP"], columns=["Sum"])

,Sum
Pred,5682.338445
FPL,5261.000000
xP,5660.700000


In [57]:
# model.predict(df[df["GW"] == GAMEWEEK_TO_PREDICT][df["Name"] == "Mohamed-Salah"][features])

In [58]:
features

['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'RD_xP',
 'xG_team_4',
 'xGA_opp_4',
 'xP_4']

In [59]:
len(features)

8

# Predictions - next gameweek only

In [60]:
preds.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,xP,Pred
83,Erling Haaland,31,2022-23,Manchester City,Leicester City,1.0,2060.531738,1722.042847,12.2,FWD,12,9.8,7.629870
197,Mohamed Salah,31,2022-23,Liverpool,Leeds United,0.0,1925.248169,1694.963013,12.8,MID,14,7.5,6.774670
152,Kevin De Bruyne,31,2022-23,Manchester City,Leicester City,1.0,2060.531738,1722.042847,12.1,MID,8,6.5,5.905531
180,Gabriel Martinelli Silva,31,2022-23,Arsenal,West Ham,0.0,1946.847900,1751.608398,6.8,MID,5,4.6,5.478469
110,Ivan Toney,31,2022-23,Brentford,Wolves,0.0,1782.324097,1717.951904,7.8,FWD,2,4.0,5.431069
244,Son Heung-min,31,2022-23,Tottenham,Bournemouth,1.0,1851.721558,1665.276123,11.6,MID,9,8.2,5.129082
211,Ollie Watkins,31,2022-23,Aston Villa,Newcastle Utd,1.0,1801.513672,1876.322510,7.6,FWD,16,10.5,4.939620
243,Solly March,31,2022-23,Brighton,Chelsea,0.0,1826.634033,1827.184204,5.2,MID,5,2.8,4.932591
34,Bruno Borges Fernandes,31,2022-23,Manchester Utd,Nott'ham Forest,0.0,1878.940186,1635.660278,9.6,MID,3,8.1,4.895869
37,Bukayo Saka,31,2022-23,Arsenal,West Ham,0.0,1946.847900,1751.608398,8.6,MID,0,7.0,4.694030


In [61]:
# preds[preds["Team"] == "Manchester City"].head(20)

# Predictions - all remaining gameweeks

In [62]:
preds_all_gameweeks.head(15)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,xP,Pred
690,Erling Haaland,35,2022-23,Manchester City,Leeds United,1.0,2069.102051,1662.209595,12.4,FWD,2,6.0,8.923433
689,Erling Haaland,34,2022-23,Manchester City,Fulham,0.0,2064.563232,1717.714355,12.4,FWD,8,9.0,8.277846
691,Erling Haaland,36,2022-23,Manchester City,Everton,0.0,2071.091064,1694.975342,12.4,FWD,7,4.6,8.149551
1590,Mohamed Salah,38,2022-23,Liverpool,Southampton,0.0,1950.095581,1616.916504,13.1,MID,5,8.6,8.028790
1589,Mohamed Salah,37,2022-23,Liverpool,Aston Villa,1.0,1955.095703,1812.286499,13.1,MID,5,3.7,7.824024
687,Erling Haaland,31,2022-23,Manchester City,Leicester City,1.0,2060.531738,1722.042847,12.2,FWD,12,9.8,7.629870
692,Erling Haaland,37,2022-23,Manchester City,Chelsea,1.0,2087.472656,1794.312012,12.4,FWD,1,1.4,7.621426
1584,Mohamed Salah,32,2022-23,Liverpool,Nott'ham Forest,1.0,1926.870483,1623.869141,12.9,MID,7,4.3,7.385410
688,Erling Haaland,33,2022-23,Manchester City,Arsenal,1.0,2057.953369,1928.582397,12.3,FWD,14,11.6,7.351327
1588,Mohamed Salah,36,2022-23,Liverpool,Leicester City,0.0,1944.225098,1709.140259,13.1,MID,12,10.2,7.274949


In [63]:
# preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Mohamed Salah"].sort_values(by=["GW"])

In [64]:
# preds_all_gameweeks[preds_all_gameweeks["Name_original"] == "Erling Haaland"].sort_values(by=["GW"])

# Saving to file

In [65]:
preds_all_gameweeks.to_csv("predictions/all_predictions.csv", index=False)

In [66]:
# pickle.dump(model, open("models/GBR.pkl", 'wb'))

# Future preds

In [67]:
df_upcoming = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

Columns (2,3,6,36,43,44,45,59,117,119,180,241) have mixed types.Specify dtype option on import or set low_memory=False.


In [68]:
df_upcoming.shape

(37410, 307)

In [69]:
df_upcoming = add_linear_fixtures(df_upcoming)

In [70]:
df_upcoming.shape

(37410, 313)

In [71]:
def adjust_goalkeepers(df, df_upcoming):
    treshold = 30
    gk_last_season = pd.read_csv('data/misc/gk_last_season.csv')
    for gk in df_upcoming[df_upcoming["FPL_pos"] == "GK"]["Name_original"].unique():
        gk_games = df[df["Name_original"] == gk].shape[0]
        if gk_games < treshold:
            current_avg = df_upcoming[df_upcoming["Name_original"] == gk].tail(1)["Avg_FPL_points"].item()
            gk_team = df_upcoming[df_upcoming["Name_original"] == gk].tail(1)["Team"].item()
            if gk_last_season[gk_last_season["team"] == current_teams_to_fpl(gk_team)].shape[0] > 0:
                last_season_avg = gk_last_season[gk_last_season["team"] == current_teams_to_fpl(gk_team)]["total_points"].item()
            else:
                last_season_avg = 3.2 # default for newly-promoted teams
            df_upcoming.loc[df_upcoming["Name_original"] == gk, "Avg_FPL_points"] = ( current_avg * gk_games + last_season_avg * (30 - gk_games) ) / 30
            # print(gk, gk_games, gk_team, current_avg, last_season_avg, current_avg * gk_games)
    return df_upcoming

In [72]:
df_upcoming = adjust_goalkeepers(df, df_upcoming)

In [73]:
# df_upcoming[df_upcoming["FPL_pos"] == "GK"].groupby(["Name_original"]).tail(1)[features + ["Name_original"]]

In [74]:
df_upcoming = df_upcoming[~df_upcoming[features].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [75]:
df_upcoming.tail(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FWD,GK,MID,Finished,Was_home_xP,RD_xP,Team_xP,Opp_xP,Avg_xP,Avg_FPL_points_venue
37405,2886,2024-04-20,NaN,NaN,Burnley,Sheffield United,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.693973,2838.312988,2717.960612,2.0,NaN
37406,2887,2024-04-27,NaN,NaN,Burnley,Manchester Utd,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.614323,2838.312988,3101.582845,2.0,NaN
37407,2758,2024-05-04,NaN,NaN,Burnley,Newcastle Utd,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,1.666667,1.612885,2838.312988,3109.506226,2.0,1.666667
37408,2888,2024-05-11,NaN,NaN,Burnley,Tottenham,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.618602,2838.312988,3078.240763,2.0,NaN
37409,2759,2024-05-19,NaN,NaN,Burnley,Nott'ham Forest,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,1.666667,1.672378,2838.312988,2812.266032,2.0,1.666667


In [76]:
df_upcoming = df_upcoming[(df_upcoming["Season"] == CURRENT_SEASON) & (df_upcoming["GW"] >= NEXT_GAMEWEEK)].reset_index(drop=True)

In [77]:
df_upcoming.shape

(16446, 313)

In [78]:
df_upcoming

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FWD,GK,MID,Finished,Was_home_xP,RD_xP,Team_xP,Opp_xP,Avg_xP,Avg_FPL_points_venue
0,1928,2023-09-24,NaN,NaN,Brighton,Bournemouth,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.555556,0.577955,1022.104763,921.671482,0.722222,0.500000
1,1954,2023-09-30,NaN,NaN,Brighton,Aston Villa,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,0.556809,1022.104763,1015.911933,0.722222,0.666667
2,1929,2023-10-08,NaN,NaN,Brighton,Liverpool,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.555556,0.542817,1022.104763,1089.631890,0.722222,0.500000
3,1955,2023-10-21,NaN,NaN,Brighton,Manchester City,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,0.531176,1022.104763,1159.640978,0.722222,0.666667
4,1930,2023-10-29,NaN,NaN,Brighton,Fulham,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.555556,0.567787,1022.104763,964.700521,0.722222,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16441,2886,2024-04-20,NaN,NaN,Burnley,Sheffield United,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.693973,2838.312988,2717.960612,2.000000,NaN
16442,2887,2024-04-27,NaN,NaN,Burnley,Manchester Utd,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.614323,2838.312988,3101.582845,2.000000,NaN
16443,2758,2024-05-04,NaN,NaN,Burnley,Newcastle Utd,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,1.666667,1.612885,2838.312988,3109.506226,2.000000,1.666667
16444,2888,2024-05-11,NaN,NaN,Burnley,Tottenham,NaN,NaN,FW,NaN,...,1.0,0.0,0.0,False,0.000000,1.618602,2838.312988,3078.240763,2.000000,NaN


In [79]:
X_test = df_upcoming

In [80]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_next_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [81]:
def adjust_first_choice_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [82]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_first_choice_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [83]:
# df_upcoming[df_upcoming["Name_original"] == "Erling Haaland"][["Name_original", "Was_home", 'Avg_FPL_points_venue']]

In [84]:
preds = get_predictions(model, df_upcoming[df_upcoming["FPL_pos"].isin(allowed_pos)], X_test[X_test["FPL_pos"].isin(allowed_pos)])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [85]:
# X_test[(X_test["GW"] == NEXT_GAMEWEEK) & (X_test["FPL_pos"] == "GK")][features + ["Name_original"]]

In [86]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Pred
4644,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2087.353760,1687.359619,14.1,FWD,NaN,8.773204
11581,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1961.337402,1807.522339,12.5,MID,NaN,6.774089
6718,James Maddison,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,7.8,MID,NaN,6.306911
6916,James Ward-Prowse,6.0,2023-24,West Ham,Liverpool,0.0,1807.522339,1961.337402,6.1,MID,NaN,5.854319
2207,Bryan Mbeumo,6.0,2023-24,Brentford,Everton,1.0,1826.335205,1682.138794,6.8,MID,NaN,5.783767
2239,Bukayo Saka,6.0,2023-24,Arsenal,Tottenham,1.0,1928.752808,1846.944458,8.7,MID,NaN,5.615363
2141,Bruno Borges Fernandes,6.0,2023-24,Manchester Utd,Burnley,0.0,1860.949707,1702.987793,8.4,MID,NaN,5.601026
14602,Solly March,6.0,2023-24,Brighton,Bournemouth,1.0,1839.788574,1659.008667,6.6,MID,NaN,5.433329
2470,Carlton Morris,6.0,2023-24,Luton,Wolves,1.0,1592.930542,1708.177246,5.5,FWD,NaN,5.273568
3755,Destiny Udogie,6.0,2023-24,Tottenham,Arsenal,0.0,1846.944458,1928.752808,4.7,DEF,NaN,5.045279


In [87]:
# preds[(preds["GW"] == NEXT_GAMEWEEK) & (preds["FPL_pos"] == "GK")].head(21)

In [88]:
n_gameweeks = 5
pivot = pd.pivot_table(preds[preds["GW"] < NEXT_GAMEWEEK + n_gameweeks], values='Pred', index=['Name_original', "FPL_pos"],
                       columns=['GW'], aggfunc=np.sum)
pivot['Summary'] = pivot.sum(axis=1)
pivot.columns = ['GW' + str(col).split(".")[0] for col in pivot.columns[:-1]] + ['Summary']
pivot = pivot.sort_values(by=["Summary"], ascending = False)
pivot.head(20)

,,GW6,GW7,GW8,GW9,GW10,Summary
Name_original,FPL_pos,,,,,,
Erling Haaland,FWD,8.773204,8.276935,7.571682,8.259087,7.728261,40.609169
James Maddison,MID,6.306911,6.579554,7.410696,7.297111,6.845971,34.440244
Mohamed Salah,MID,6.774089,6.315755,6.359743,7.187918,7.157520,33.795025
James Ward-Prowse,MID,5.854319,7.189914,6.403089,6.195052,6.991331,32.633705
Destiny Udogie,DEF,5.045279,5.537355,6.074511,6.075316,5.552227,28.284688
Bukayo Saka,MID,5.615363,5.833824,4.976261,5.473790,6.283082,28.182319
Bruno Borges Fernandes,MID,5.601026,5.698622,5.514948,5.745926,4.933613,27.494135
Jarrod Bowen,MID,4.796727,6.051102,5.348417,5.099895,5.872922,27.169063
Bryan Mbeumo,MID,5.783767,5.424142,4.988185,5.698063,5.172347,27.066503


In [89]:
pivot[pivot.index.get_level_values('FPL_pos') == 'GK']

,,GW6,GW7,GW8,GW9,GW10,Summary
Name_original,FPL_pos,,,,,,
Guglielmo Vicario,GK,4.796574,4.990005,5.570413,5.354562,5.309229,26.020783
Alisson Ramses Becker,GK,4.391863,4.237692,4.219349,4.643763,4.464301,21.956968
José Malheiro de Sá,GK,4.493015,3.526266,4.112628,4.462438,4.130059,20.724405
Emiliano Martínez Romero,GK,3.404990,3.475271,3.688335,3.614857,3.830839,18.014292
Norberto Murara Neto,GK,3.265476,3.245399,3.669482,3.838756,3.857010,17.876123
Mark Flekken,GK,3.813691,3.534012,3.122637,3.723347,3.537170,17.730856
Nick Pope,GK,3.494954,3.594512,3.286571,3.621250,3.396003,17.393289
Ederson Santana de Moraes,GK,3.684008,3.490190,3.021640,3.462808,3.115023,16.773668
Aaron Ramsdale,GK,3.315018,3.602961,2.797520,3.156988,3.586070,16.458557


In [90]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,264.664213
James Maddison,228.494449
Mohamed Salah,221.433534
James Ward-Prowse,214.768853
Destiny Udogie,189.013786
Bukayo Saka,185.604981
Bruno Borges Fernandes,182.030297
Jarrod Bowen,179.042013
Guglielmo Vicario,173.090505


In [91]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Feature importance and influence

In [92]:
# explainer = shap.Explainer(model.model_outfield.predict, X_test[2000:3000])
# shap_values = explainer(X_test[2000:3000])

In [93]:
# shap.plots.bar(shap_values, max_display=15)

In [94]:
# shap.plots.beeswarm(shap_values, max_display=15)

In [95]:
# # explaining Erling Haaland`s score in gameweek 31
# shap.plots.bar(shap_values[69], max_display=15)

In [96]:
# shap.plots.waterfall(shap_values[410])

In [97]:
# shap.plots.waterfall(shap_values[26])

In [98]:
# shap.plots.waterfall(shap_values[96])